## Some global config

In [59]:
import requests
import pandas as pd
import os
import time
from config import *
from bs4 import BeautifulSoup
import json

## 新出行
API list：

In [ ]:
api_post = 'https://xchuxing.com/evaluation?page={}'
api_post_detail = 'https://xchuxing.com/article/{}'
api_cmt = 'https://xchuxing.com/comments/{}/1/1'

SITE = 'xchuxing'
PAGE_NUM = 30
SAVE_DIR = os.path.join(DATA_PATH, TASKS[0], SITE)

- headers

In [ ]:
HEADER = {
    'Content-Type': 'application/json;charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Accept': 'application/json, text/javascript, */*; q=0.01s',
    'Cookie': '_identity-frontend=848196bf64e754c33d8fa582494ed42147f04110774c851088e6eb8770b4e134a%3A2%3A%7Bi%3A0%3Bs%3A18%3A%22_identity-frontend%22%3Bi%3A1%3Bs%3A50%3A%22%5B76367%2C%22okTqBUaPh2Zvl_LL3bPN0zz4LD_CdKuN%22%2C2592000%5D%22%3B%7D; xcx-frontend=0ejmssd6ed0lafeq6g4s83ialp; Hm_lvt_55f107d825a4c4e989d543a8bf935508=1648210339,1648894971,1648911715,1648911780; Hm_lpvt_55f107d825a4c4e989d543a8bf935508=1648912533',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_POST = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cookie': '_identity-frontend=848196bf64e754c33d8fa582494ed42147f04110774c851088e6eb8770b4e134a%3A2%3A%7Bi%3A0%3Bs%3A18%3A%22_identity-frontend%22%3Bi%3A1%3Bs%3A50%3A%22%5B76367%2C%22okTqBUaPh2Zvl_LL3bPN0zz4LD_CdKuN%22%2C2592000%5D%22%3B%7D; xcx-frontend=0ejmssd6ed0lafeq6g4s83ialp; Hm_lvt_55f107d825a4c4e989d543a8bf935508=1648210339,1648894971,1648911715,1648911780; Hm_lpvt_55f107d825a4c4e989d543a8bf935508=1648912533',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

- request

In [ ]:
csv_post, csv_cmt = {}, {}
for key in ['tid', 'title', 'summary', 'commentnum', 'category_id', 'category_name', 'create_time', 'content']:
    csv_post[key] = list()
for key in ['id', 'tid', 'content', 'liketimes', 'updated_at']:
    csv_cmt[key] = list()

for page in range(1, PAGE_NUM+1):
    resp_sum = requests.get(api_post.format(page), headers=HEADER).json()
    for sdata in resp_sum['data']:
        tid = sdata['tid']
        
        keys = list(csv_post.keys())
        keys.remove('content')
        for key in keys:
            csv_post[key].append(sdata[key]) if sdata.get(key) else csv_post[key].append('0')
        
        # content
        post_html = requests.get(api_post_detail.format(tid), headers=HEADER_POST).content.decode()
        soup = BeautifulSoup(post_html)
        content = [s for s in soup.article.stripped_strings]
        content = ''.join(content[4:-5])        
        csv_post['content'].append(content)
        
        resp_cmt = requests.get(api_cmt.format(tid), headers=HEADER).json()
        for cdata in resp_cmt['data']:
            for key in csv_cmt.keys():
                if cdata.get(key):
                    csv_cmt[key].append(cdata[key]) if key != 'updated_at' else csv_cmt[key].append(time.strftime('%Y-%m-%d', time.localtime(cdata[key])))
                else:
                    csv_cmt[key].append('0')

- save csv file

In [ ]:
df_sum, df_cmt = pd.DataFrame(csv_post), pd.DataFrame(csv_cmt)
os.makedirs(SAVE_DIR, exist_ok=True)
df_sum.to_csv(os.path.join(SAVE_DIR, 'posts.csv'))
df_cmt.to_csv(os.path.join(SAVE_DIR, 'comments.csv'))

## 电动邦
API list：

In [ ]:
api_post = 'https://webapi.diandong.com/content/list?page={}&size=15&source_id=12&content_type=all'
api_post_detail = 'https://www.diandong.com/news/{}.html'
api_cmt = 'https://openapi.diandong.com/comment/list/{}/1/50'

SITE = 'diandongbang'
PAGE_NUM = 330
SAVE_DIR = os.path.join(DATA_PATH, TASKS[0], SITE)

- headers

In [ ]:
HEADER_POSTS = {
    'content-encoding': 'gzip',
    'content-type': 'text/html; charset=UTF-8',
    # 'Accept': 'application/json, text/javascript, */*; q=0.01s',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_DETAIL = {
    'content-type': 'text/html; charset=utf8',
    'content-encoding': 'gzip',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_CMT = {
    'Content-Type': 'application/json;charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Accept': 'application/json, text/javascript, */*; q=0.01s',
    'Cookie': '_identity-frontend=848196bf64e754c33d8fa582494ed42147f04110774c851088e6eb8770b4e134a%3A2%3A%7Bi%3A0%3Bs%3A18%3A%22_identity-frontend%22%3Bi%3A1%3Bs%3A50%3A%22%5B76367%2C%22okTqBUaPh2Zvl_LL3bPN0zz4LD_CdKuN%22%2C2592000%5D%22%3B%7D; xcx-frontend=0ejmssd6ed0lafeq6g4s83ialp; Hm_lvt_55f107d825a4c4e989d543a8bf935508=1648210339,1648894971,1648911715,1648911780; Hm_lpvt_55f107d825a4c4e989d543a8bf935508=1648912533',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

- request

In [ ]:
csv_post, csv_cmt = {}, {}
for key in ['catid', 'contentid', 'title', 'tags', 'published', 'author', 'comment_num', 'data_type', 'content']:
    csv_post[key] = list()
for key in ['replyid', 'sourceid', 'content', 'authorid', 'author', 'supports', 'showTime']:
    csv_cmt[key] = list()

for page in range(248, PAGE_NUM+1):
    resp_sum = requests.get(api_post.format(page), headers=HEADER_POSTS).json()
    for sdata in resp_sum['data']['list']:
        if sdata['data_type'] == 'video':
            continue
        
        tid = sdata['contentid']
        
        keys = list(csv_post.keys())
        keys.remove('content')
        for key in keys:
            csv_post[key].append(sdata[key]) if sdata.get(key) else csv_post[key].append('0')
        
        # content
        post_html = requests.get(api_post_detail.format(tid), headers=HEADER_DETAIL).content.decode()
        soup = BeautifulSoup(post_html)
        content = [s for s in soup.find_all("div", class_="detail-article-content my-gallery")[0].stripped_strings]        
        csv_post['content'].append(content)
        
        resp_cmt = requests.get(api_cmt.format(tid), headers=HEADER_CMT).json()
        for cdata in resp_cmt['data']['reply']:
            for key in csv_cmt.keys():
                if cdata.get(key):
                    csv_cmt[key].append(cdata[key])
                else:
                    csv_cmt[key].append('0')

- save csv file

In [ ]:
df_sum, df_cmt = pd.DataFrame(csv_post), pd.DataFrame(csv_cmt)
os.makedirs(SAVE_DIR, exist_ok=True)
df_sum.to_csv(os.path.join(SAVE_DIR, 'posts.csv'))
df_cmt.to_csv(os.path.join(SAVE_DIR, 'comments.csv'))

## 车云

- api

In [ ]:
api_post = 'https://openapi.cheyun.com/article/list/0/46343%7C46245%7C46279%7C46309%7C46305%7C46331%7C46330%7C46320%7C46316%7C46362%7C46360%7C46355%7C46354%7C46329%7C46325%7C46324%7C46303%7C46229%7C46227%7C/{}/{}'
api_post_detail = 'https://openapi.cheyun.com/article/{}'
api_cmt = 'http://m.cheyun.com/comment/get_comment?page=1&zw_id={}&length=100'

SITE = 'cheyun'
PAGE_NUM, PAGE_SIZE = 30, 100
SAVE_DIR = os.path.join(DATA_PATH, TASKS[0], SITE)

- headers

In [ ]:
HEADER_POSTS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_CMT = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_DETAIL = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}

- request

In [ ]:
csv_post, csv_cmt = {}, {}
for key in ['id', 'title', 'description', 'tags', 'inputtime', 'type', 'author', 'type', 'content']:
    csv_post[key] = list()
for key in ['id', 'user_id', 'content', 'ding', 'cai', 'create_time', 'update_time']:
    csv_cmt[key] = list()

for page in range(29, PAGE_NUM+1):
    resp_sum = requests.get(api_post.format(page, PAGE_SIZE), headers=HEADER_POSTS).json()
    for sdata in resp_sum['data']['list']:
        tid = sdata['id']
        
        keys = list(csv_post.keys())
        keys.remove('content')
        for key in keys:
            csv_post[key].append(sdata[key]) if sdata.get(key) else csv_post[key].append('0')
        
        # content
        post_html = requests.get(api_post_detail.format(tid), headers=HEADER_DETAIL).json()['data']['content']
        soup = BeautifulSoup(post_html)
        content = '\n'.join([s for s in soup.stripped_strings] )
        csv_post['content'].append(content)
        
        resp_cmt = requests.get(api_cmt.format(tid), headers=HEADER_CMT).json()
        for cdata in resp_cmt['msg']['msg']:
            for key in csv_cmt.keys():
                if cdata.get(key):
                    csv_cmt[key].append(cdata[key])
                else:
                    csv_cmt[key].append('0')

- save csv file

In [ ]:
df_sum, df_cmt = pd.DataFrame(csv_post), pd.DataFrame(csv_cmt)
os.makedirs(SAVE_DIR, exist_ok=True)
df_sum.to_csv(os.path.join(SAVE_DIR, 'posts.csv'))
df_cmt.to_csv(os.path.join(SAVE_DIR, 'comments.csv'))

## 汽车之家

- api

In [ ]:
api_post = 'https://www.autohome.com.cn/ashx/v2/bestauto/getBastautoContentFlow?search_after={}'
api_post_detail = 'https://www.autohome.com.cn/drive/202204/1243748-all.html?pvareaid={}#'
api_cmt = 'https://reply.autohome.com.cn/ShowReply/ReplyTopLevelList.ashx?_appid=m&appid=1&objid={}&next=0&pagesize=50&datatype=json&ordertype=0&replydata=1'

SITE = 'autohome'
PAGE_NUM, PAGE_SIZE = 30, 100
SAVE_DIR = os.path.join(DATA_PATH, TASKS[0], SITE)

- header

In [ ]:
HEADER_POSTS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_CMT = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_DETAIL = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'upgrade-insecure-requests': '1',
    'sec-fetch-dest': 'document',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}

- request

In [ ]:
csv_post, csv_cmt = {}, {}
for key in ['biz_id', 'title', 'small_title', 'publish_time', 'author_name', 'reply_count', 'content']:
    csv_post[key] = list()
for key in ['RMemberId', 'RMemberName', 'RObjId', 'RContent', 'replydate']:
    csv_cmt[key] = list()

END_TIME = '2021/01/01'
search_after = ''
stime = '2022/4/30'

while(stime > END_TIME):
    resp_sum = requests.get(api_post.format(search_after), headers=HEADER_POSTS).json()
    search_after = resp_sum['message']
    stime = resp_sum['result']['items'][0]['publish_time'].split(' ')[0]

    for sdata in resp_sum['result']['items']:
        tid = sdata['biz_id']
        keys = list(csv_post.keys())
        keys.remove('content')
        for key in keys:
            csv_post[key].append(sdata[key]) if sdata.get(key) else csv_post[key].append('0')
        
        # content
        post_html = requests.get(api_post_detail.format(tid), headers=HEADER_DETAIL).content.decode('gb2312')
        soup = BeautifulSoup(post_html)
        content = '\n'.join([s for s in soup.stripped_strings] )
        csv_post['content'].append(content)
        
        resp_cmt = requests.get(api_cmt.format(tid), headers=HEADER_CMT).json()
        if not resp_cmt['result'].get('list'):
            continue

        for cdata in resp_cmt['result']['list']:
            for key in csv_cmt.keys():
                if cdata.get(key):
                    csv_cmt[key].append(cdata[key])
                else:
                    csv_cmt[key].append('0')

- save file

In [ ]:
df_sum, df_cmt = pd.DataFrame(csv_post), pd.DataFrame(csv_cmt)
os.makedirs(SAVE_DIR, exist_ok=True)
df_sum.to_csv(os.path.join(SAVE_DIR, 'posts.csv'))
df_cmt.to_csv(os.path.join(SAVE_DIR, 'comments.csv'))

## 新车评

In [ ]:
api_post = 'https://www.xincheping.com/pingce/firsttry/p{}.html'
api_post_detail = 'https://www.xincheping.com/pingce/{}.html'
api_cmt = 'https://www.xincheping.com/cmt/list.do?targetId={}&targetType=1&pageNo={}'

SITE = 'xincheping'
SAVE_DIR = os.path.join(DATA_PATH, TASKS[0], SITE)

END_TIME = '2018/01/01'
stime = '2022/04/30'

- Header

In [ ]:
HEADER_POST = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_CMT = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_DETAIL = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}

- helper functions

In [ ]:
def filt_post_id(htm):
    ks, kd = 'https://www.xincheping.com/pingce/', '.html'
    soup = BeautifulSoup(htm)
    cps = soup.find_all('div', class_='cp_fbox')
    
    data = list()

    for cp in cps:
        data.append(
            {
                'tid': cp.a['href'].replace(ks,'').replace(kd,''),
                'title': cps[0].h2.string,
                'author': list(cp.span.stripped_strings)[-2],
                'create_time': list(cp.span.stripped_strings)[-1]
            }
        )
    
    return data

def filt_post_content(htm):
    soup = BeautifulSoup(htm)
    soup = soup.find('div', class_='de_info')
    content = '\n'.join([s for s in soup.stripped_strings] )
    return content

- requests

In [ ]:
csv_post, csv_cmt = {}, {}
for key in ['tid', 'title', 'author', 'create_time', 'content']:
    csv_post[key] = list()
for key in ['targetId', 'userId', 'userName', 'cmtContent', 'time']:
    csv_cmt[key] = list()
PN = 1

while(stime > END_TIME):
    htm = requests.get(api_post.format(PN), headers=HEADER_POST).content.decode()
    resp_sum = filt_post_id(htm)
    
    stime = resp_sum[0]['create_time']
    PN += 1

    for sdata in resp_sum:
        tid = sdata['tid']
        keys = list(csv_post.keys())
        keys.remove('content')
        for key in keys:
            csv_post[key].append(sdata[key]) if sdata.get(key) else csv_post[key].append('0')
        
        # content
        post_html = requests.get(api_post_detail.format(tid), headers=HEADER_DETAIL).content.decode()
        csv_post['content'].append(filt_post_content(post_html))
        
        page_num = 1
        while(1):
            resp_cmt = requests.get(api_cmt.format(tid, page_num), headers=HEADER_CMT).json()
            
            if not resp_cmt['result']['item'].get('newCmtList'):
                break
            data = resp_cmt['result']['item']
            for cdata in data['newCmtList']:
                csv_cmt['targetId'].append(cdata['targetId'])
                csv_cmt['time'].append(cdata['time'])
                csv_cmt['cmtContent'].append(cdata['cmtContent'])
                
                csv_cmt['userId'].append(cdata['userInfo']['userId'])
                csv_cmt['userName'].append(cdata['userInfo']['userName'])

            page_num += 1

- save to file

In [ ]:
df_sum, df_cmt = pd.DataFrame(csv_post), pd.DataFrame(csv_cmt)
os.makedirs(SAVE_DIR, exist_ok=True)
df_sum.to_csv(os.path.join(SAVE_DIR, 'posts.csv'))
df_cmt.to_csv(os.path.join(SAVE_DIR, 'comments.csv'))

## 车质网

- api list

In [163]:
api_post = 'http://www.12365auto.com/scbg/jp-0-0-0--1-0-0-0-0-{}.shtml'
api_post_detail = 'http://www.12365auto.com/scbg/{}/{}-all.shtml'
api_cmt = 'http://comment.12365auto.com/server/getComment.ashx?_callback=jQuery17206171924443972485_1650694101492&replyid={}&type=1&act=getAjaxComment&size=50&page=1&zjdy=0&_=1650694102465'

SITE = '12365auto'
SAVE_DIR = os.path.join(DATA_PATH, TASKS[0], SITE)

END_TIME = '2019-00-00 00:00'
stime = '2022-04-30 00:00'

- headers

In [164]:
HEADER_POST = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_CMT = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_DETAIL = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}

In [165]:
def filt_post_id(htm):
    soup = BeautifulSoup(htm)
    pcs = list(soup.find_all('div', class_='list_show'))
    titles = [pc.h1.find_all('a')[-1].string for pc in pcs]
    ids = [pc.find('div', class_='p_left').a['href'].split('/')[-1].split('.')[0] for pc in pcs]
    tis = [pc.find('div', class_='p_left').a['href'].split('/')[-2] for pc in pcs]
    data = []
    for title, id, ti in zip(titles, ids, tis):
        data.append(
            {
                'tid': id,
                'title': title,
                'ti': ti
            }
        )
    return data

def filt_post_content(htm):
    soup = BeautifulSoup(htm)
    soup = soup.find('div', class_='pjnr')

    info = list(soup.find('div', class_='lef').stripped_strings)
    frm = info[0].split('：')[-1]
    time = info[1].split('：')[-1]
    author = info[2].split('：')[-1]
    editor = info[3].split('：')[-1]

    content = ''.join(soup.find('div', class_='show').stripped_strings)

    data = {
        'from': frm,
        'create_time': time,
        'author': author,
        'editor': editor,
        'content': content
    }

    return data

def JQ2JS(cnt):
    cnt = requests.get(url=api_cmt.format(473524), headers=HEADER_CMT).content
    cnt = cnt.decode()
    cnt = cnt[cnt.find('(')+1:cnt.rfind(')')]
    return json.loads(cnt.encode())

- request

In [170]:
csv_post, csv_cmt = {}, {}
for key in ['tid', 'ti', 'title','from', 'create_time', 'author', 'editor', 'content']:
    csv_post[key] = list()
for key in ['p_id', 'replyid', 'p_username', 'p_up', 'p_date', 'p_content', 'replyid']:
    csv_cmt[key] = list()

PN = 1

while(stime > END_TIME):
    htm = requests.get(url=api_post.format(PN), headers=HEADER_POST).content.decode()
    resp_sum = filt_post_id(htm)
    PN += 1

    for sdata in resp_sum:
        tid = sdata['tid']
        ti = sdata['ti']
        keys = list(sdata.keys())

        if requests.get(api_post_detail.format(ti,tid), headers=HEADER_DETAIL).status_code == 404:
            continue

        for key in keys:
            csv_post[key].append(sdata[key]) if sdata.get(key) else csv_post[key].append('0')
        
        # content & author
        post_html = requests.get(api_post_detail.format(ti, tid), headers=HEADER_DETAIL).content.decode('gbk')
        pdata = filt_post_content(post_html)
        stime = pdata['create_time']

        for key in pdata.keys():
            csv_post[key].append(pdata[key])

        cnt = requests.get(api_cmt.format(tid), headers=HEADER_CMT).content
        resp_cmt = JQ2JS(cnt)
        keys = list(csv_cmt.keys())
        keys.remove('replyid')

        for cdata in resp_cmt['cmntlist']:
            csv_cmt['replyid'].append(tid)
            for key in keys:
                csv_cmt[key].append(cdata[key])


- save to file

In [171]:
df_sum, df_cmt = pd.DataFrame(csv_post), pd.DataFrame(csv_cmt)
os.makedirs(SAVE_DIR, exist_ok=True)
df_sum.to_csv(os.path.join(SAVE_DIR, 'posts.csv'))
df_cmt.to_csv(os.path.join(SAVE_DIR, 'comments.csv'))

## 第一电动

In [95]:
api_post = 'https://www.d1ev.com/carnews/pingce/list-{}'
api_post_detail = 'https://www.d1ev.com/carnews/pingce/{}'
api_cmt = 'https://www.d1ev.com/contents/comment/get_comment?per_page=1&id={}&type=10'

SITE = 'd1ev'
SAVE_DIR = os.path.join(DATA_PATH, TASKS[0], SITE)

END_TIME = '2018-12-30 00:00:00'
stime = '2022-04-23 00:00:00'

- header

In [96]:
HEADER_POST = {
    'Access-Control-Allow-Headers': 'Content-Type,Access-Token,Appid,Secret,Authorization,x-csrf-token,x-requested-with',
    'Access-Control-Allow-Methods': 'GET,POST,OPTIONS',
    'Connection': 'keep-alive',
    'Content-Encoding': 'gzip',
    'Content-Type': 'text/html; charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
}

HEADER_CMT = {
    'Content-Type': 'application/json;charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Accept': 'application/json, text/javascript, */*; q=0.01s',
    'Cookie': '_identity-frontend=848196bf64e754c33d8fa582494ed42147f04110774c851088e6eb8770b4e134a%3A2%3A%7Bi%3A0%3Bs%3A18%3A%22_identity-frontend%22%3Bi%3A1%3Bs%3A50%3A%22%5B76367%2C%22okTqBUaPh2Zvl_LL3bPN0zz4LD_CdKuN%22%2C2592000%5D%22%3B%7D; xcx-frontend=0ejmssd6ed0lafeq6g4s83ialp; Hm_lvt_55f107d825a4c4e989d543a8bf935508=1648210339,1648894971,1648911715,1648911780; Hm_lpvt_55f107d825a4c4e989d543a8bf935508=1648912533',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

HEADER_DETAIL = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}

- helper function

In [97]:
def filt_post_id(htm):
    soup = BeautifulSoup(htm)
    cps = soup.find_all('div', class_='article--content am-fl')
    titles =[list(cp.a.stripped_strings)[0] for cp in cps]
    times = [list(cp.time.stripped_strings)[0] for cp in cps]
    ids = [cp.a['href'].split('/')[-1] for cp in cps]
    
    data = list()

    for id, time, title in zip(ids, times, titles):
        data.append(
            {
                'tid': id,
                'title': title,
                'create_time': time
            }
        )
    
    return data

def filt_post_content(htm):
    soup = BeautifulSoup(htm)
    author = list(soup.find('div', class_='ws-title').a.stripped_strings)[-1]
    content = '\n'.join(list(soup.find('div', class_='ws-newscon').stripped_strings))

    return author, content

- request

In [147]:
csv_post, csv_cmt = {}, {}
for key in ['tid', 'title', 'author', 'create_time', 'content']:
    csv_post[key] = list()
for key in ['id', 'targetId', 'addtime', 'userId', 'nickName', 'content', 'isHot']:
    csv_cmt[key] = list()
PN = 1

while(stime > END_TIME):
    htm = requests.get(url=api_post.format(PN), headers=HEADER_POST).content.decode()
    resp_sum = filt_post_id(htm)
    stime = resp_sum[0]['create_time']
    PN += 1

    for sdata in resp_sum:
        tid = sdata['tid']
        keys = list(sdata.keys())

        if requests.get(api_post_detail.format(tid), headers=HEADER_DETAIL).status_code == 404:
            continue

        for key in keys:
            csv_post[key].append(sdata[key]) if sdata.get(key) else csv_post[key].append('0')
        
        # content & author
        post_html = requests.get(api_post_detail.format(tid), headers=HEADER_DETAIL).content.decode()
        author, content = filt_post_content(post_html)
        csv_post['content'].append(content)
        csv_post['author'].append(author)

        resp_cmt = requests.get(api_cmt.format(tid), headers=HEADER_CMT).content
        
        resp_cmt = json.loads(resp_cmt)

        for cdata in resp_cmt['comment']:
            csv_cmt['id'].append(cdata['id'])
            csv_cmt['addtime'].append(cdata['addtime'])
            csv_cmt['userId'].append(cdata['userId'])
            csv_cmt['nickName'].append(cdata['nickName'])
            csv_cmt['content'].append(cdata['content'])
            csv_cmt['isHot'].append(cdata['isHot'])
            
            csv_cmt['targetId'].append(resp_cmt['article']['targetId'])

KeyError: 'create_time'

- save to file

In [99]:
df_sum, df_cmt = pd.DataFrame(csv_post), pd.DataFrame(csv_cmt)
os.makedirs(SAVE_DIR, exist_ok=True)
df_sum.to_csv(os.path.join(SAVE_DIR, 'posts.csv'))
df_cmt.to_csv(os.path.join(SAVE_DIR, 'comments.csv'))